In [ ]:
import cv2
import os
import numpy as np
import face_recognition
import pandas as pd
from datetime import datetime
import tkinter as tk
from tkinter import simpledialog, messagebox
from PIL import Image, ImageTk

# Folder to store images
IMAGE_FOLDER = "registered_faces"
if not os.path.exists(IMAGE_FOLDER):
    os.makedirs(IMAGE_FOLDER)

# Attendance file
ATTENDANCE_FILE = "attendance.csv"

class FaceRecognitionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Face Recognition Attendance")
        self.root.geometry("800x500")
        self.root.configure(bg="white")

        # Video frame
        self.video_label = tk.Label(self.root)
        self.video_label.pack(side=tk.LEFT, padx=10, pady=10)

        # Button frame
        button_frame = tk.Frame(self.root, bg="white")
        button_frame.pack(side=tk.RIGHT, padx=20, pady=20)

        tk.Button(button_frame, text="Register", command=self.capture_image, width=20, height=2).pack(pady=10)
        tk.Button(button_frame, text="Mark Attendance", command=self.mark_attendance, width=20, height=2).pack(pady=10)
        tk.Button(button_frame, text="Delete ID", command=self.manage_ids, width=20, height=2).pack(pady=10)
        tk.Button(button_frame, text="Exit", command=self.root.quit, width=20, height=2, bg="red", fg="white").pack(pady=10)

        self.cap = cv2.VideoCapture(0)
        self.update_video()

    def update_video(self):
        ret, frame = self.cap.read()
        if ret:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (400, 300))
            img = Image.fromarray(frame)
            imgtk = ImageTk.PhotoImage(image=img)
            self.video_label.imgtk = imgtk
            self.video_label.config(image=imgtk)
        self.root.after(10, self.update_video)

    def capture_image(self):
        ret, frame = self.cap.read()
        if ret:
            name = simpledialog.askstring("Input", "Enter Name:")
            if name:
                file_path = f"{IMAGE_FOLDER}/{name}.jpg"
                cv2.imwrite(file_path, frame)
                messagebox.showinfo("Success", f"{name} registered successfully!")

    def encode_faces(self):
        encodings, names = [], []
        for file in os.listdir(IMAGE_FOLDER):
            img_path = os.path.join(IMAGE_FOLDER, file)
            img = cv2.imread(img_path)
            if img is None:
                continue
            rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            encoding = face_recognition.face_encodings(rgb_img)
            if encoding:
                encodings.append(encoding[0])
                names.append(os.path.splitext(file)[0])
        return encodings, names

    def mark_attendance(self):
        known_encodings, known_names = self.encode_faces()
        ret, frame = self.cap.read()
        if ret:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            face_locations = face_recognition.face_locations(rgb_frame)
            face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

            for encoding in face_encodings:
                matches = face_recognition.compare_faces(known_encodings, encoding)
                face_distances = face_recognition.face_distance(known_encodings, encoding)
                best_match_index = np.argmin(face_distances) if matches else -1

                if best_match_index != -1 and matches[best_match_index]:
                    name = known_names[best_match_index]
                    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                    date = datetime.now().strftime('%Y-%m-%d')

                    # Check if the file exists and has content
                    if not os.path.exists(ATTENDANCE_FILE) or os.stat(ATTENDANCE_FILE).st_size == 0:
                        df = pd.DataFrame([[1, name, date, timestamp]], columns=["Sl.No", "Name", "Date", "Time"])
                        df.to_csv(ATTENDANCE_FILE, index=False)
                    else:
                        # Read existing data
                        df = pd.read_csv(ATTENDANCE_FILE)

                        # Add new entry with incremental Sl.No
                        new_entry = pd.DataFrame([[len(df) + 1, name, date, timestamp]], columns=df.columns)
                        df = pd.concat([df, new_entry], ignore_index=True)

                        # Save back to CSV
                        df.to_csv(ATTENDANCE_FILE, index=False)

                    messagebox.showinfo("Attendance Marked", f"Attendance recorded for {name}")
                else:
                    messagebox.showwarning("Unregistered", "Person not registered!")


    def manage_ids(self):
        name = simpledialog.askstring("Manage ID", "Enter Name to Delete:")
        if name:
            file_path = os.path.join(IMAGE_FOLDER, f"{name}.jpg")
            if os.path.exists(file_path):
                action = messagebox.askquestion("Manage ID", "Do you want to delete this ID?", icon='warning')
                if action == 'yes':
                    os.remove(file_path)
                    messagebox.showinfo("Success", f"{name} deleted successfully!")
            else:
                messagebox.showerror("Error", "Name not found!")

if __name__ == "__main__":
    root = tk.Tk()
    app = FaceRecognitionApp(root)
    root.mainloop()
